In [1]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
import yfinance as yf

yf.pdr_override()

In [2]:
tickers = ['PG', 'BEI.DE']

sec_data = pd.DataFrame()

for t in tickers:
    sec_data[t] = wb.DataReader(t, start='2007-1-1')['Adj Close']

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [3]:
sec_data.tail()

,PG,BEI.DE
Date,,
2023-12-26,145.940002,NaN
2023-12-27,146.059998,134.600006
2023-12-28,145.729996,135.000000
2023-12-29,146.539993,135.699997
2024-01-02,148.740005,135.000000


In [4]:
sec_returns = np.log(sec_data / sec_data.shift(1))

In [5]:
sec_returns

,PG,BEI.DE
Date,,
2007-01-03,NaN,NaN
2007-01-04,-0.007621,0.006544
2007-01-05,-0.008624,-0.020772
2007-01-08,0.002203,0.000202
2007-01-09,-0.002517,-0.022858
...,...,...
2023-12-26,0.004533,NaN
2023-12-27,0.000822,NaN
2023-12-28,-0.002262,0.002967


## PG

In [6]:
sec_returns['PG'].mean()

0.00030947464667356155

In [7]:
sec_returns['PG'].mean() * 250

0.07736866166839039

In [8]:
sec_returns['PG'].std()

0.011729162772708758

In [9]:
sec_returns['PG'].std() * 250 ** 0.5

0.18545434704307756

## Beiersdorf

In [10]:
sec_returns['BEI.DE'].mean()

0.0002676515167112688

In [11]:
sec_returns['BEI.DE'].mean() * 250

0.0669128791778172

In [12]:
sec_returns['BEI.DE'].std()

0.013568664819104436

In [13]:
sec_returns['BEI.DE'].std() * 250 ** 0.5

0.21453942817883292

***

In [14]:
print (sec_returns['PG'].mean() * 250)
print (sec_returns['BEI.DE'].mean() * 250)

0.07736866166839039
0.0669128791778172


In [16]:
sec_returns[['PG', 'BEI.DE']].mean() * 250

PG        0.077369
BEI.DE    0.066913
dtype: float64

In [17]:
sec_returns[['PG', 'BEI.DE']].mean() * 250

PG        0.077369
BEI.DE    0.066913
dtype: float64

In [18]:
sec_returns[['PG', 'BEI.DE']].std() * 250 ** 0.5

PG        0.185454
BEI.DE    0.214539
dtype: float64

## Covariance and Correlation


\begin{eqnarray*}
Covariance Matrix: \  \   
\Sigma = \begin{bmatrix}
        \sigma_{1}^2 \ \sigma_{12} \ \dots \ \sigma_{1I} \\
        \sigma_{21} \ \sigma_{2}^2 \ \dots \ \sigma_{2I} \\
        \vdots \ \vdots \ \ddots \ \vdots \\
        \sigma_{I1} \ \sigma_{I2} \ \dots \ \sigma_{I}^2
    \end{bmatrix}
\end{eqnarray*}

In [19]:
PG_var = sec_returns['PG'].var()
PG_var

0.000137573259348697

In [20]:
BEI_var = sec_returns['BEI.DE'].var()
BEI_var

0.00018410866497320242

In [21]:
PG_var_a = sec_returns['PG'].var() * 250
PG_var_a

0.03439331483717425

In [22]:
BEI_var_a = sec_returns['BEI.DE'].var() * 250
BEI_var_a

0.0460271662433006

***

In [23]:
cov_matrix = sec_returns.cov()
cov_matrix

,PG,BEI.DE
PG,0.000138,0.000043
BEI.DE,0.000043,0.000184


In [24]:
cov_matrix_a = sec_returns.cov() * 250
cov_matrix_a

,PG,BEI.DE
PG,0.034393,0.010695
BEI.DE,0.010695,0.046027


***

In [25]:
corr_matrix = sec_returns.corr()
corr_matrix

,PG,BEI.DE
PG,1.000000,0.268171
BEI.DE,0.268171,1.000000


## Calculating Portfolio Risk

Equal weigthing scheme:

In [26]:
weights = np.array([0.5, 0.5])

Portfolio Variance:

In [27]:
pfolio_var = np.dot(weights.T, np.dot(sec_returns.cov() * 250, weights))
pfolio_var

0.025452446423597706

Portfolio Volatility:

In [28]:
pfolio_vol = (np.dot(weights.T, np.dot(sec_returns.cov() * 250, weights))) ** 0.5
pfolio_vol

0.15953822872151271

In [29]:
print (str(round(pfolio_vol, 5) * 100) + ' %')

15.953999999999999 %


## Calculating Diversifiable and Non-Diversifiable Risk of a Portfolio

In [30]:
weights = np.array([0.5, 0.5])

In [31]:
weights[0]

0.5

In [32]:
weights[1]

0.5

***

Diversifiable Risk:

In [33]:
PG_var_a = sec_returns[['PG']].var() * 250
PG_var_a

PG    0.034393
dtype: float64

In [34]:
BEI_var_a = sec_returns[['BEI.DE']].var() * 250
BEI_var_a

BEI.DE    0.046027
dtype: float64

In [35]:
dr = pfolio_var - (weights[0] ** 2 * PG_var_a) - (weights[1] ** 2 * BEI_var_a)
dr

BEI.DE   NaN
PG       NaN
dtype: float64

In [36]:
float(PG_var_a)

0.03439331483717425

In [37]:
PG_var_a = sec_returns['PG'].var() * 250
PG_var_a

0.03439331483717425

In [38]:
BEI_var_a = sec_returns['BEI.DE'].var() * 250
BEI_var_a

0.0460271662433006

In [39]:
dr = pfolio_var - (weights[0] ** 2 * PG_var_a) - (weights[1] ** 2 * BEI_var_a)
dr

0.005347326153478995

In [40]:
print (str(round(dr*100, 3)) + ' %')

0.535 %


Non-Diversifiable Risk:

In [41]:
n_dr_1 = pfolio_var - dr
n_dr_1

0.02010512027011871

In [42]:
n_dr_2 = (weights[0] ** 2 * PG_var_a) + (weights[1] ** 2 * BEI_var_a)
n_dr_2

0.020105120270118713

In [43]:
n_dr_1 == n_dr_2

False